In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

from deep_translator import GoogleTranslator

In [27]:
titels =[]

adresses = []
ratings = []
reviews_list = []

positive = []
neutral = []
negative = []

scores_sent = []

In [28]:
url = "https://www.google.com/maps"

driver = webdriver.Edge()

driver.get(url)

accept_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(2) > div > div > button"))
)

accept_button.click()

search = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#searchboxinput"))
)
search.send_keys("Restaurants Luxembourg city")
search.send_keys(Keys.RETURN)
time.sleep(10)


#elems = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')

for i in range(0):
    driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
elems = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".Nv2PK.THOPZb.CpccDe"))
)
print("Number of feed elements found:", len(elems))
for elem in elems:
    title = elem.find_element(By.CSS_SELECTOR, "div.fontHeadlineSmall")
    titels.append(title.text)

links = driver.find_elements(By.CLASS_NAME, "hfpxzc")

links_list = [link.get_attribute("href") for link in links]

driver.close()
print(titels)

Number of feed elements found: 9
['Bar en Bistro Raposa', 'Grootmoeders Keuken', 'Pitta Maestro', 'Tokyo Sushi', "'t Stoveke", 'Pizza Santa Melle', "'t Boerderijtje", 'RESTO PASSIVE PROGRAM', "'t Fornuizeke"]


In [29]:

#links_list = ["https://www.google.com/maps/place/Bar+en+Bistro+Raposa/@51.0021478,3.7975561,17z/data=!3m1!4b1!4m6!3m5!1s0x47c375196eb5d1cf:0xdac616be092e319a!8m2!3d51.0021478!4d3.7975561!16s%2Fg%2F11hdmy4jmr?authuser=0&hl=nl&entry=ttu"]
for url in links_list:
    driver = webdriver.Edge()
    driver.get(str(url))
    #Wait for accept cookies button and press accept
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#yDmH0d > c-wiz > div > div > div > div.NIoIEf > div.G4njw > div.AIC7ge > div.CxJub > div.VtwTSb > form:nth-child(2) > div > div > button"))
    )
    accept_button.click()
    time.sleep(2)
    #Get address
    try:
        adress = driver.find_element(By.CSS_SELECTOR, 'div.Io6YTe.fontBodyMedium.kR99db')
        adresses.append(adress.text)
    except:
        print("Error getting address")
        adresses.append("None")
    #Get rating
    try:
        rating = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]')
        ratings.append(rating.text)
    except:
        print("Error getting rating")
        ratings.append("None")
    # #Get reviews
    # reviews_button = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)'))
    # )
    # reviews_button.click()
    # time.sleep(5)
    # for i in range(5):
    #     driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]').send_keys(Keys.PAGE_DOWN)
    #     time.sleep(1)
    # reviews = driver.find_elements(By.CSS_SELECTOR, 'span.wiI7pd')
    # print("Number of feed elements found:", len(reviews))
    # for elem in reviews:
    #     reviews_list.append(elem.text)
    driver.quit()


Error getting rating
Error getting rating


In [30]:
print(len(ratings))
print(len(adresses))
print(ratings)
print(adresses)

9
9
['4,6', 'None', '3,3', '4,5', '4,5', '4,5', '4,7', 'None', '4,6']
['Vossenstraat 2, 9090 Melle', 'Guldensporenlaan 9, 9820 Merelbeke', 'Brusselsesteenweg 200, 9090 Melle', 'Brusselsesteenweg 277, 9090 Melle', 'Oude Geraardsbergse Steenweg 26, 9090 Melle', 'Gemeenteplein 24, 9090 Melle', 'Wellingstraat 95, 9070 Destelbergen', 'Gontrode Heirweg, 9090 Melle', 'Mellestraat 286, 9090 Melle']


In [31]:
    # en_review_list = []
    # for element in reviews_list:
    #     translated = GoogleTranslator(source='auto', target='en').translate(text=element)
    #     en_review_list.append(translated)
    
    # for element in en_review_list:
    #     def preprocess(text):
    #         new_text = []
    #         for t in text.split(" "):
    #             t = '@user' if t.startswith('@') and len(t) > 1 else t
    #             t = 'http' if t.startswith('http') else t
    #             new_text.append(t)
    #         return " ".join(new_text)
    #     MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    #     tokenizer = AutoTokenizer.from_pretrained(MODEL)
    #     config = AutoConfig.from_pretrained(MODEL)
    #     model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    #     #model.save_pretrained(MODEL)
    #     text = element
    #     text = preprocess(text)
    #     encoded_input = tokenizer(text, return_tensors='pt')
    #     output = model(**encoded_input)
    #     scores = output[0][0].detach().numpy()
    #     scores = softmax(scores)

    #     ranking = np.argsort(scores)
    #     #ranking = ranking[::-1]
    #     for i in range(scores.shape[0]):
    #         l = config.id2label[ranking[i]]
    #         s = scores[ranking[i]]
    #         print(f"{i+1}) {l} {np.round(float(s), 4)}")
    #     # positive.append(scores[ranking[2]])
    #     # neutral.append(scores[ranking[1]])
    #     # negative.append(scores[ranking[0]])
    #     scores_sent.append(ranking)

In [35]:
import pandas as pd
df = pd.DataFrame(
    {'Name': titels,
     'Link': links_list,
     'Address': adresses,
     'Rating': ratings

    })

In [38]:
df.head()
df.to_csv("file.csv", index=False)

In [34]:
df_reviews = pd.DataFrame(
    {'Review': en_review_list,
     'Positive': positive,
     'Neutral': neutral,
     'Negative': negative
    })

ValueError: All arrays must be of the same length

In [ ]:
df_reviews.to_csv("test.csv")